In [1]:
# =============================================================================
# Probability quick notes 
# -----------------------------------------------------------------------------
# RANDOM VARIABLE (RV)
# - A (measurable) map X : (Omega, F, P) -> R.
#   * Omega = sample space (all outcomes)
#   * F     = sigma-algebra of events
#   * P     = probability measure on (Omega, F)
# - Range/image of X is a subset of R.
# - Support = points in R where the distribution has nonzero mass/density.
#
# CDF
# - F_X(t) = P(X <= t).
# - Always exists (discrete, continuous, or mixed).
#
# CONTINUOUS CASE: DENSITY (pdf)
# - If X has a pdf f(x), then P(a <= X <= b) = ∫[a,b] f(x) dx, and ∫ f(x) dx = 1.
# - Histogram estimator (iid samples x_1,...,x_N):
#     height(bin around x) = (count_in_bin) / (N * bin_width)
#   Under regularity, as N -> ∞, bin_width h -> 0, and N*h -> ∞,
#   histogram height at the bin midpoint approximates f(x).
#
# DISCRETE CASE: PROBABILITY MASS FUNCTION (pmf)
# - p_X(x) = P(X = x), with p_X(x) >= 0 and sum_x p_X(x) = 1.
# - CDF: F_X(t) = sum_{x <= t} p_X(x) (for ordered support).
# - Estimation from data: p_hat(x) = (# of samples equal to x) / N.
#   NOTE: Unlike histograms for densities, DO NOT divide by bin_width.
#
# EXPECTATION / VARIANCE
# - Continuous: E[X] = ∫ x f(x) dx; Var(X) = ∫ (x - E[X])^2 f(x) dx.
# - Discrete:   E[X] = sum_x x p_X(x); Var(X) = sum_x (x - E[X])^2 p_X(x).
#
# MIXED DISTRIBUTIONS
# - Some variables have both discrete atoms and continuous parts.
#   * A histogram captures only the continuous component.
#   * Atoms appear as spikes that grow as bin_width -> 0.
# - The empirical CDF F_N(t) = (1/N) * sum 1{x_i <= t} always converges
#   uniformly to F_X(t) (Glivenko–Cantelli), regardless of type.
#
# PRACTICAL HISTOGRAM TIP
# - For a density-like histogram in code: use density=True in plotting, or
#   manually set heights to (counts / (N * bin_width)) to approximate pdf.
# =============================================================================


In [2]:
# --- Uniform Monte Carlo (baseline) ---
# Goal: estimate I = ∫_0^1 f(x) dx when the target density is uniform t(x) = 1 on [0,1]
# Draw X ~ Unif(0,1). By the Law of Large Numbers, the sample mean → expected value as N → ∞:
#     I = ∫_0^1 f(x) t(x) dx = E_t[f(X)]  ≈  (1/N) ∑_{i=1}^N f(X_i),   X_i ~ Unif(0,1)
# This estimator is simple and unbiased. Its variance is:
#     Var(Ī_MC) = (1/N) Var_t(f(X)) = (1/N) [ E_t(f(X)^2) - (E_t[f(X)])^2 ] = (1/N) [E_t(f(X)^2) - I^2] = (1/N) [ ∫_0^1 f(x)^2 dx − I^2] (high variance if f varies a lot over [0,1])

# --- Importance Sampling (variance reduction) ---
# Idea: sample from a proposal pdf p(x) on [0,1] that places more mass where |f(x)| is large, then reweight to keep the same target integral
# Let w(x) = t(x)/p(x). Since t(x)=1 on [0,1]:
#     I = ∫ f(x) t(x) dx = ∫ f(x) w(x) p(x) dx = E_{X~p}[ f(X) w(X) ]
# Monte Carlo estimator:
#     Ī_IS = (1/N) ∑ f(X_i) w(X_i),   X_i ~ p
# Variance (assuming normalized t and p):
#     Var(Ī_IS) = (1/N) [ E_p( (f(X) w(X))^2 ) - I^2 ] = (1/N) [ ∫ f(x)^2 t(x)^2 / p(x) dx - I^2 ] (if p ~ |f| (or |f|·t), typically lower variance than uniform MC)

# --- Inversion Sampling (to draw from nonuniform distribution p(x) on [0,1]) ---
# If U ~ Unif(0,1) and F_p is a CDF with inverse F_p^{-1}, then X = F_p^{-1}(U) has PDF p(x) = dF_p/dx
# Conversely, given p(x), compute its CDF F_p(x) = ∫_0^x p(t) dt, then invert to get F_p^{-1} (this may be analytical or numerical)

In [3]:
#  --- Metropolis/Hastings Algorithm ---
#  Goal: Generate a Markov chain {x_0, x_1, …, x_T} whose marginal distribution converges to the target distribution p(x) on [a, b]
#  (use an unnormalized density \tilde{p}(x) ∝ p(x) because algorithm only uses ratios)

# Initialization:
# 1) Choose an initial distribution μ₀ on [a,b] and draw x₀ ~ μ₀ (ensure \tilde{p}(x₀) > 0)
# 2) Choose a proposal kernel q(·|x): a normalized probability distribution on any subset of ℝ, given the current state x (e.g., Uniform(x-δ, x+δ))
#    q(y|x) = transition probability density of proposing y given current state x
# 3) Set the number of time steps T (and optionally a burn-in and thinning)
#
# Loop (for t = 0,1,2,…,T-1):
# 1) Draw y ~ q(·|x_t) (propose new state y given current state x_t)
# 2) Compute the acceptance ratio
#       α(x_t, y) = min{ 1,[ \tilde{p}(y) / \tilde{p}(x_t) ] * [ q(x_t|y) / q(y|x_t) ] }.
#    For *symmetric* proposals (e.g., Uniform(x±δ), Normal(x, s²)), the Hastings factor q(x_t|y)/q(y|x_t) = 1
#
# 3) Accept/Reject:
#    - If α = 1 (i.e., weight ≥ 1): set x_{t+1} = y. (always accept)
#    - Else if 0 < α < 1 (i.e., 0 < weight < 1): draw r ~ Uniform(0,1)
#         If r ≤ α (i.e., r ≤ weight): set x_{t+1} = y (accept); else x_{t+1} = x_t (reject)
#    - If α = 0 (e.g., y outside support so \tilde{π}(y)=0): set x_{t+1} = x_t
#
# Convergence (intuition):
#   For large T (after optional discarding first several samples (burning) and keeping every k-th sample (thinning)), the remaining states {x_t} are approximately distributed according to π
#   (though the {x_t} are correlated, not i.i.d.)
#
# Tips for choosing q(·|x):
# - Support/Reachability: proposals should allow reaching any region where π>0
# - Symmetric, simple samplers (Uniform window or Gaussian RW) keep code easy because the q-ratio cancels
# - Tune step size (δ or s) to target ~40–60% acceptance in 1D
#      - Too small δ: very high acceptance, but tiny moves → strong autocorrelation → inefficient (slow mixing); also may not explore full support for finite T
#      - Too large δ: very low acceptance → many rejections → poor exploration (also inefficient)
# - Boundaries: proposing on ℝ and auto-rejecting out-of-bounds is simplest; or truncate/renormalize q near edges (then include q-ratio)


In [4]:
# 2D Ising Model Simulation

In [ ]:
# leapfrog/CTCS scheme for the 1D damped wave equation: u_tt + 2k u_t = c^2 u_xx  (here c = 1)
    # --------------------------------------------------------------------------------------------
    # u[j,i] = u(x_i, t_j) (initialize to zero) with j,i = 0,1,...,nt-1,nx-1 as indices for time and space (coordinate-index convention)
    # goal: compute time step t_j -> t_{j+1} for j = 0,..., nt-2 at interior spatial points x_i for i = 1,..., nx-2 (since u at boundaries already known)
    #
    # approximate derivatives using first and second-order *central* differences at (x_i, t_j):
    #   u_tt(x_i, t_j) ≈ [ u(x_i, t_{j+1}) - 2 u(x_i, t_j) + u(x_i, t_{j-1}) ] / Δt^2                   
    #   u_t (x_i, t_j) ≈ [ u(x_i, t_{j+1}) - u(x_i, t_{j-1}) ] / (2 Δt)                                 
    #   u_xx(x_i, t_j) ≈ [ u(x_{i+1}, t_j) - 2 u(x_i, t_j) + u(x_{i-1}, t_j) ] / Δx^2                   
    #
    # substitute approximations into u_tt + 2k u_t = u_xx and rearrange to solve for u(x_i, t_{j+1}):
    #   u(x_i, t_{j+1}) ≈ [ 2 u(x_i, t_j) - (1 - k Δt) u(x_i, t_{j-1}) + r^2 ( u(x_{i+1}, t_j) - 2 u(x_i, t_j) + u(x_{i-1}, t_j) ) ] / (1 + k Δt)
    # -->
    #   u[j+1,i] ≈ [2*u[j,i] - (1 - k*Δt)*u[j-1,i] + (r**2)*(u[j,i+1] - 2*u[j,i] + u[j,i-1])]/(1 + k*Δt)
    #
    # leapfrog needs two past time levels (j and j-1), so we must build j = 1 row via a Taylor expansion about t = 0:
    #   u(x_i, Δt) ≈ u(x_i, 0) + Δt u_t(x_i, 0) + (1/2) Δt^2 u_tt(x_i, 0) with initial data u(x,0) = f(x) and u_t(x,0) = g(x)
    # from the PDE u_tt + 2 k u_t = u_xx, at t = 0, u_tt(x_i, 0) = u_xx(x_i, 0) - 2 k u(x_i, 0) = u_xx(x_i, 0) - 2 k g(x_i)
    # approximate u_xx(x_i, 0) ≈ [ u(x_{i+1}, 0) - 2 u(x_i, 0) + u(x_{i-1}, 0) ] / Δx^2 (second-order central difference), yielding:
    #   u(x_i, Δt) ≈ u(x_i, 0) + Δt g(x_i) + (1/2) Δt^2 [ ( u(x_{i+1}, 0) - 2 u(x_i, 0) + u(x_{i-1}, 0) ) / Δx^2 - 2 k * g(x_i)]
    # -->
    #  u[1,i] ≈ u[0,i] + Δt*g(xs[i]) + (1/2)*(Δt**2)*((u[0,i+1] - 2*u[0,i] + u[0,i-1])/Δx**2 - 2*k*g(xs[i]))


In [ ]:
# FTCS scheme for time-dependent 2D Schrödinger equation (ℏ =  1, m = 1/2) : i ∂_t ψ(x, y, t) = H ψ(x, y, t),  with  H = -Δ + V(x, y),   Δ = ∂_xx + ∂_yy
    # ---------------------------------------------------------------------------------------------------------------------------------------------------------
    # goal: compute time step t_k -> t_{k+1} for k = 0,..., nt-2 at interior spatial points x_i, y_j for i,j = 1,..., nx-2, ny-2 (since ψ at boundaries already known)
    #
    # approximate ∂_t ψ at (x_i, y_j, t_k) using *forward* difference in time:
    #   ∂_t ψ(x_i, y_j, t_k) ≈ ∂_t ψ(x_i, y_j, t_k) ≈ [ ψ(x_i, y_j, t_{k+1}) − ψ(x_i, y_j, t_k) ] / Δt
    #   -->
    #   [ ψ(x_i, y_j, t_{k+1}) − ψ(x_i, y_j, t_k) ] / Δt ≈ − i H ψ(x_i, y_j, t_k)                        
    #
    # approximate Laplacian at (x_i, y_j, t_k) using second-order *central* difference in space (denote this discrete Laplacian operator as Δ_h):
    #   Δψ(x_i,y_j,t_k) ≈ [ψ(x_{i+1}, y_j, t_k) - 2 ψ(x_i, y_j, t_k) + ψ(x_{i-1},y_j,t_k)] / Δx^2 + [ψ(x_i, y_{j+1}, t_k) - 2 ψ(x_i, y_j, t_k) + ψ(x_i,y_{j-1},t_k)] / Δy^2
    #   
    # substitute approximations into H = - Δ + V and rearrange to solve for ψ(x_i, y_j, t_{k+1}):
    #   ψ(x_i, y_j, t_{k+1}) ≈ ψ(x_i, y_j, t_k) - i Δt [ - Δ_h ψ(x_i, y_j, t_k) + V(x_i, y_j) ψ(x_i, y_j, t_k) ]
    #
    # real/imaginary split: ψ = R + i I, with R(x_i,y_j,t_k) = Re ψ(x_i,y_j,t_k), I(x_i,y_j,t_k) = Im ψ(x_i,y_j,t_k)
    # -->
    #   I(x_i, y_j, t_{k+1}) ≈ I(x_i, y_j, t_k) + dt ( Δ_h R(x_i, y_j, t_k) - V(x_i, y_j) R(x_i, y_j, t_k) ) (compute I at t_{k+1} using R at t_k)
    #   R(x_i, y_j, t_{k+1}) ≈ R(x_i, y_j, t_k) - dt ( Δ_h I(x_i, y_j, t_k+1) - V(x_i, y_j) I(x_i, y_j, t_k+1) ) (compute R at t_{k+1} using previously computed I at t_{k+1}; this is a symplectic update which is more stable than a pure FTCS)
    #
    # I[j,i,k] = I(x_i, y_j, t_k) and R[j,i,k] = R(x_i, y_j, t_k) with j,i,k = 0,...,ny-1,nx-1,nt-1 as indices for y,x,t respectively (coordinate-index convention)
    # -->
    #   I[j,i,k+1] ≈ I[j,i,k] + r_x*(R[j,i+1,k] - 2*R[j,i,k] + R[j,i-1,k]) + r_y*(R[j+1,i,k] - 2*R[j,i,k] + R[j-1,i,k]) - dt*V[j,i]*R[j,i,k]
    #   R[j,i,k+1] ≈ R[j,i,k] - r_x*(I[j,i+1,k+1] - 2*I[j,i,k+1] + I[j,i-1,k+1]) - r_y*(I[j+1,i,k+1] - 2*I[j,i,k+1] + I[j-1,i,k+1]) + dt*V[j,i]*I[j,i,k+1]
    #
    # probability (norm) remark:
    #   exact step (unitary):
    #     ψ(k+1) = U(Δt) ψ(k),  with  U(Δt) = exp(−i H Δt),  U†U = I  --> discrete probability |ψ(k)|^2 = ∑_{i,j} |ψ{j,i,k}|^2 Δx Δy is constant in k (= 1 if normalized)
    #
    #   FTCS approximate step:
    #     ψ(k+1) ≈ (1 - i Δt H) ψ(k) = ψ(k) - i Δt H ψ(k) --> [ ψ(k+1) - ψ(k) ]/ Δt ≈ - i H ψ(k) (same as above)
    #     not unitary: the step operator I − i Δt H is not unitary, so discrete norm/probability drifts over time: 
    #     |ψ(k+1)|^2 = <ψ(k+1), ψ(k+1)> ≈ <(I - i Δt H) ψ(k), (I - i Δt H) ψ(k)> = <ψ(k), (I + i Δt H)(I - i Δt H) ψ(k)> = <ψ(k), (I + (Δt)^2 H^2) ψ(k)> 
    #                = |ψ(k)|^2 + (Δt)^2 <ψ(k), H^2 ψ(k)>  = |ψ(k)|^2 + O((Δt)^2)
    #     so discrete probability drifts by O((Δt)^2) each step, and thus by O(Δt) over finite time intervals (# of steps = lt/Δt) 
    #     however, since we take very small Δt, this drift is negligible over the full time scale (approximately unitary behavior observed in practice) 